<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/chat_bot_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/weedge/chat-bot.git

Cloning into 'chat-bot'...
remote: Enumerating objects: 954, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 954 (delta 182), reused 264 (delta 107), pack-reused 570
Receiving objects: 100% (954/954), 4.25 MiB | 4.16 MiB/s, done.
Resolving deltas: 100% (400/400), done.


In [1]:
%cd chat-bot

/content/chat-bot


In [4]:
!git checkout fix/remote-queue-chat

Branch 'fix/remote-queue-chat' set up to track remote branch 'fix/remote-queue-chat' from 'origin'.
Switched to a new branch 'fix/remote-queue-chat'


### install dependencies

In [3]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [5]:
!apt-get install python3-pyaudio && pip -q install pyannote.audio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 python3-pyaudio
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 91.2 kB of archives.
After this operation, 340 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 91.2 kB in 1s (63.5 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package python3-pyaudio.
Prep

In [6]:
!pip install -q pyaudio pydub redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 19.8 MB/s eta 0:00:00


In [7]:
# asr whisper
!pip install -q openai-whisper whisper-timestamped faster-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 42.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00

In [8]:
# llamacpp https://github.com/abetlen/llama-cpp-python?tab=readme-ov-file#installation
# cpu
#!pip install llama-cpp-python
#!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
#!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

# gpu
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.80-cp310-cp310-linux_x86_64.whl size=176752545 sha256=a147178bc46ea296e42ad16892313ddc8e97fdd5eefc294df0fda7c9d4e21545
  Stored in directory: /root/.cache/pip/wheels/06/d7/73/c60eef37c97fd8a469541258e318e0c566cc23b97b3aaa2b08
Successfully built llama-cpp-python


In [ ]:
# ChatTTS
!git submodule update --init --recursive
!pip install -q vocos
!pip install vector_quantize_pytorch

In [9]:
# CoquiTTS
!pip install -U TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 80.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 108.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 121.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 103.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 71.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other

In [ ]:
!pip install numpy==1.23

In [5]:
!pip install TTS

In [6]:
!pip list | grep torch

pytorch-lightning                2.3.1
pytorch-metric-learning          2.5.0
torch                            2.3.0+cu121
torch-audiomentations            0.11.1
torch-pitch-shift                1.2.4
torchaudio                       2.3.0+cu121
torchmetrics                     1.4.0.post0
torchsummary                     1.5.1
torchtext                        0.18.0
torchvision                      0.18.0+cu121


In [7]:
!pip show omegaconf einops IPython ctranslate2 numpy TTS

Name: omegaconf
Version: 2.3.0
Summary: A flexible configuration library
Home-page: https://github.com/omry/omegaconf
Author: Omry Yadan
Author-email: omry@yadan.net
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: antlr4-python3-runtime, PyYAML
Required-by: pyannote.audio
---
Name: einops
Version: 0.8.0
Summary: A new flavour of deep learning operations
Home-page: https://github.com/arogozhnikov/einops
Author: Alex Rogozhnikov
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: encodec, pyannote.audio, TTS
---
Name: ipython
Version: 7.34.0
Summary: IPython: Productive Interactive Computing
Home-page: https://ipython.org
Author: The IPython Development Team
Author-email: ipython-dev@python.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: backcall, decorator, jedi, matplotlib-inline, pexpect, pickleshare, prompt-toolkit, pygments, setuptools, traitlets
Required-by: cufflinks, datascience, 

### download models

In [12]:
# asr openai whisper
!huggingface-cli download openai/whisper-base  --local-dir ./models/openai/whisper-base --local-dir-use-symlinks False
# faster-whisper
!huggingface-cli download Systran/faster-whisper-base  --local-dir ./models/Systran/faster-whisper-base --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-medium  --local-dir ./models/Systran/faster-whisper-medium --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-large-v2  --local-dir ./models/Systran/faster-whisper-large-v2 --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-distil-whisper-large-v3  --local-dir ./models/Systran/faster-distil-whisper-large-v3 --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 16 files: 100% 16/16 [00:00<00:00, 2091.07it/s]
/content/chat-bot/models/openai/whisper-base
/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 6 files: 100% 6/6 [00:00<00:00, 10437.92it/s]
/content/chat-bot/models/Systran/faster-whisper-base
/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]Downloading 'tokenizer.json' to 'models/Systran/faster-whisper-medium/.huggingface/download/tokenizer.json.7

In [9]:
# llm llamacpp phi-3-mini-4k-instruct
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-q4.gguf --local-dir ./models --local-dir-use-symlinks False
# llm llamacpp Qwen2-1.5B-Instruct
!huggingface-cli download Qwen/Qwen2-1.5B-Instruct-GGUF qwen2-1_5b-instruct-q8_0.gguf  --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Phi-3-mini-4k-instruct-q4.gguf: 100% 2.39G/2.39G [00:09<00:00, 246MB/s]
Download complete. Moving file to models/Phi-3-mini-4k-instruct-q4.gguf
models/Phi-3-mini-4k-instruct-q4.gguf
/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
qwen2-1_5b-instruct-q8_0.gguf: 100% 1.65G/1.65G [01:15<00:00, 21.9MB/s]
Download complete. Moving file to models/qwen2-1_5b-instruct-q8_0.gguf
models/qwen2-1_5b-instruct-q8_0.gguf


In [ ]:
# tts chatTTS
!huggingface-cli download 2Noise/ChatTTS  --local-dir ./models/2Noise/ChatTTS --local-dir-use-symlinks False

In [10]:
# tts coquiTTS
!huggingface-cli download coqui/XTTS-v2  --local-dir ./models/coqui/XTTS-v2 --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 18 files:   0% 0/18 [00:00<?, ?it/s]Downloading 'hash.md5' to 'models/coqui/XTTS-v2/.huggingface/download/hash.md5.0342b1a8e1708abdf53a86af8eb252762a6b0b79.incomplete'

hash.md5: 100% 32.0/32.0 [00:00<00:00, 248kB/s]
Download complete. Moving file to models/coqui/XTTS-v2/hash.md5

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 14.1MB/s]
Download complete. Moving file to models/coqui/XTTS-v2/.gitattributes
Fetching 18 files:   6% 1/18 [00:00<00:13,  1.31it/s]
LICENSE.txt: 100% 4.01k/4.01k [00:00<00:00, 31.0MB/s]

README.md:   0% 0.00/4.26k [00:00<?, ?B/s]Download complete. Moving file to models/coqui/XTTS-v2/LICENSE.txt
README.md: 100% 4.26k/4.26k [00:00<00:00, 21.1MB/s]
Download complete. Moving file to models/coqui/XTTS-v2/README.md

config.json: 100% 4.37k/4.37k [00

### run chat bot BE

In [13]:
from google.colab import userdata
redis_pwd = userdata.get('REDIS_PASSWORD')

In [14]:
!mkdir -p log

In [ ]:
!REDIS_PASSWORD=$redis_pwd RUN_OP=be TQDM_DISABLE=True python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [15]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen2-1_5b-instruct-q8_0.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.
start loop_asr_llm_generate with whisper_faster_asr llm_llamacpp qwen ...
start loop_tts_synthesize with tts_coqui ...
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/chat-bot/src/cmd/remote-queue-chat/generate_au